In [9]:
import pandas as pd
import numpy as np
import osmnx as ox
import matplotlib.pyplot as plt
from geopy.geocoders import Nominatim
from geopy.exc import GeocoderTimedOut
import time
import geopandas as gpd
from shapely.geometry import Polygon, MultiPoint , Point
from math import ceil
import matplotlib.pyplot as plt
import seaborn as sns
from shapely.geometry import Polygon, MultiPoint , Point
from shapely import wkt
import textgenrnn
from textgenrnn import textgenrnn
from scipy.sparse import random
from scipy.optimize import linear_sum_assignment



/Users/erensmacbook/opt/anaconda3/lib/python3.9/site-packages/keras/optimizer_v2/adam.py:105: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(Adam, self).__init__(name, **kwargs)


In [15]:
#functions
def plot_dist(train_data_1,synthetic_data_cop):

    dist_a = train_data_1.groupby('INDUSTRY').size()
    dist_b = synthetic_data_cop.groupby('INDUSTRY').size()
    
    ax = plt.subplot(111)
    index=dist_a.keys().to_list()

    w=0.3
    bar1=np.arange(len(index))
    bar2= [i+w for i in bar1]

    ax.bar(x=bar1 ,height=list(dist_a.values/dist_a.sum()), width=0.4, color='orange',label='real')
    ax.bar(x=bar2 ,height=list(dist_b.values/dist_b.sum()), width=0.4, color='green',label='fake')

    ax.autoscale(tight=True)
    
    ind=list(dist_a.keys())
    plt.xticks(range(len(ind)),ind)


    plt.legend()
    plt.show()
    


In [ ]:
def plot_distance_c(train_data_1,syn_data,title):    
    
    anchor= train_data_1.distance.max()
    plt.figure(figsize=(12,6))
    plt.xlim(0,round(anchor,3))

    plt.xlabel('Distance')
    plt.ylabel('Records')
    plt.title(title)

    sns.distplot(train_data_1['distance'],bins=30,hist_kws=dict(alpha=0.3),label='real',norm_hist=True)
    sns.distplot(syn_data['distance'],bins=30,hist_kws=dict(alpha=0.3),label='fake',norm_hist=True,color='orange')

    plt.legend()
    plt.show()

In [ ]:
def plot_area_valid(real,fake,sector,title,thresh):
    
    hungariandest=fake.dest_sa2.unique()
    realdest=real.dest_sa2.unique()
    indices=np.intersect1d(hungariandest,realdest)
    
    hungarian_educ=[]
    realdest_educ=[]
    
    ctgan_states = fake.groupby(['dest_sa2','INDUSTRY'])['INDUSTRY'].count()
    # Change: groupby state_office and divide by sum
    ctgan_pcts = ctgan_states.groupby(level=0).apply(lambda x:
                                                 100 * x / float(x.sum()))


    for i in indices:
        try:
            hungarian_educ.append(ctgan_pcts[i][sector])
        except KeyError:
            hungarian_educ.append(0)
        
    for i in indices:
        try:
            realdest_educ.append(validation[validation.dest_sa2==i][sector][0]*100)
        except KeyError:
            realdest_educ.append(0)

    sns.scatterplot( x=realdest_educ,  y=hungarian_educ)
    plt.xlim(-1, thresh)
    plt.ylim(-1, thresh)
    xpoints = ypoints = plt.xlim()
    plt.plot(xpoints, ypoints, linestyle='--', color='k', lw=3, scalex=True, scaley=True)
    plt.xlabel("Validation percentages per SA2")
    plt.ylabel("Generated percentages per SA2")
    plt.title(title)
    
    
    

In [ ]:
def generate_distance(syn_data):
    syn_data = pd.merge(syn_data,qld_orig, on="ORIGSA1_2021")
    syn_data = pd.merge(syn_data,qld_dest, on="DESTSA1_2021")
    
    syn_data['xo_c'] = syn_data["geo_o"].apply(lambda x: (x.bounds[0]+x.bounds[2]) /2)
    syn_data['yo_c'] = syn_data["geo_o"].apply(lambda x: (x.bounds[1]+x.bounds[3]) /2)

    syn_data['xd_c'] = syn_data["geo_d"].apply(lambda x: (x.bounds[0]+x.bounds[2]) /2)
    syn_data['yd_c'] = syn_data["geo_d"].apply(lambda x: (x.bounds[1]+x.bounds[3]) /2)
    
    syn_data['orig_pt']=syn_data.apply(lambda x: Point((x['xo_c'],x['yo_c'])) ,axis=1 )
    syn_data['dest_pt']=syn_data.apply(lambda x: Point((x['xd_c'],x['yd_c'])) ,axis=1 )
    syn_data['distance']=syn_data.apply(lambda x: (x['orig_pt'].distance(x['dest_pt'])) ,axis=1 )
    
    return syn_data

In [46]:
def generate_merger(df):
    
    merger=df.copy()
    
    merger['geo_0']=merger['0'].map(area_dict2)
    merger['geo_1']=merger['1'].map(area_dict2)
    merger['geo_2']=merger['2'].map(area_dict2)
    merger['geo_3']=merger['3'].map(area_dict2)

    merger = merger.dropna().reset_index()

    merger['xo_0'] = merger["geo_0"].apply(lambda x: (x.bounds[0]+x.bounds[2]) /2 if x !=0 else 0 )
    merger['yo_0'] = merger["geo_0"].apply(lambda x: (x.bounds[1]+x.bounds[3]) /2 if x !=0 else 0 )

    merger['xo_1'] = merger["geo_1"].apply(lambda x: (x.bounds[0]+x.bounds[2]) /2 if x !=0 else 0 )
    merger['yo_1'] = merger["geo_1"].apply(lambda x: (x.bounds[1]+x.bounds[3]) /2 if x !=0 else 0 )

    merger['xo_2'] = merger["geo_2"].apply(lambda x: (x.bounds[0]+x.bounds[2]) /2 if x !=0 else 0 ) 
    merger['yo_2'] = merger["geo_2"].apply(lambda x: (x.bounds[1]+x.bounds[3]) /2 if x !=0 else 0 )

    merger['xo_3'] = merger["geo_3"].apply(lambda x: (x.bounds[0]+x.bounds[2]) /2 if x !=0 else 0 )
    merger['yo_3'] = merger["geo_3"].apply(lambda x: (x.bounds[1]+x.bounds[3]) /2 if x !=0 else 0 )

    merger['pt_0']=merger.apply(lambda x: Point((x['xo_0'],x['yo_0'])) ,axis=1 )
    merger['pt_1']=merger.apply(lambda x: Point((x['xo_1'],x['yo_1'])) ,axis=1 )
    merger['pt_2']=merger.apply(lambda x: Point((x['xo_2'],x['yo_2'])) ,axis=1 )
    merger['pt_3']=merger.apply(lambda x: Point((x['xo_3'],x['yo_3'])) ,axis=1 )

    merger['distance01']=merger.apply(lambda x: 0 if x['pt_1'] ==  Point(0 ,0) else (x['pt_0'].distance(x['pt_1'])) ,axis=1 )
    merger['distance02']=merger.apply(lambda x: 0 if x['pt_2'] ==  Point(0 ,0) else (x['pt_0'].distance(x['pt_2'])) ,axis=1 )
    merger['distance03']=merger.apply(lambda x: 0 if x['pt_3'] ==  Point(0 ,0) else (x['pt_0'].distance(x['pt_3'])) ,axis=1 )
    
    return merger



In [ ]:
def plot_area(real,fake,sector,title,thresh):
    hungariandest=fake.dest_sa2.unique()
    realdest=real.dest_sa2.unique()
    indices=np.intersect1d(hungariandest,realdest)
    
    hungarian_educ=[]
    realdest_educ=[]
    
    ctgan_states = fake.groupby(['dest_sa2','INDUSTRY'])['INDUSTRY'].count()
    # Change: groupby state_office and divide by sum
    ctgan_pcts = ctgan_states.groupby(level=0).apply(lambda x:
                                                 100 * x / float(x.sum()))


    for i in indices:
        try:
            hungarian_educ.append(ctgan_pcts[i][sector])
        except KeyError:
            hungarian_educ.append(0)
        
    for i in indices:
        try:
            realdest_educ.append(real_pcts[i][sector])
        except KeyError:
            realdest_educ.append(0)

    sns.scatterplot( x=realdest_educ,  y=hungarian_educ)
    plt.xlim(-1, thresh)
    plt.ylim(-1, thresh)
    xpoints = ypoints = plt.xlim()
    plt.plot(xpoints, ypoints, linestyle='--', color='k', lw=3, scalex=False, scaley=False)
    plt.title(title)

In [ ]:
def add_sa2(hungarian15k):
    hungarian15k=hungarian15k.rename(columns={ "DESTSA1_2021":"dest_sa","ORIGSA1_2021":"orig_sa"})


    hungarian15k['SA1_CODE21']=hungarian15k['orig_sa']
    hungarian15k=pd.merge(hungarian15k,sa1_sa2, on='SA1_CODE21')


    hungarian15k=hungarian15k.rename(columns={"SA2_CODE21":"orig_sa2"})
    hungarian15k = hungarian15k.drop(['SA1_CODE21'],axis=1)
    hungarian15k['SA1_CODE21']=hungarian15k['dest_sa']

    hungarian15k=pd.merge(hungarian15k,sa1_sa2, on='SA1_CODE21')
    hungarian15k=hungarian15k.rename(columns={"SA2_CODE21":"dest_sa2"})
    hungarian15k = hungarian15k.drop(['SA1_CODE21'],axis=1)
    
    return hungarian15k

#plot_area(real_cat,hungarian50k,'health','10k kd tree education')


In [ ]:
def add_distance(merger):
    merg_dist= merger[['distance01','distance02','distance03']].idxmax(axis = 1)

    distances =[]
    sas=[]
    for i in range(len(merger)):
        indeks=merg_dist[i]

        if indeks == 'distance01':
            distances.append(merger['distance01'].iloc[i])
            sas.append(merger['1'].iloc[i])

        elif indeks =='distance02':
            distances.append(merger['distance02'].iloc[i])
            sas.append(merger['2'].iloc[i])
    
        elif indeks=='distance03':
            distances.append(merger['distance03'].iloc[i])
            sas.append(merger['3'].iloc[i])
            
            
    merger['distance']=distances
    merger['dist_sa']=sas

In [47]:
def assign_toct(d1,ctgan_sample,rnn_sample):
    dene = ctgan_sample.copy()
    dene['indexes'] = list(range(0,len(rnn_sample)))
    dene['indexes']=dene['indexes'].map(d1)
    hungarian_indices=dene['indexes'].to_list()
    ct_dests=[]
    ct_origs=[]
    for i in hungarian_indices:
        ct_dests.append(rnn_sample['dest_sa'].iloc[i])
        ct_origs.append(rnn_sample['0'].iloc[i])

    dene['dest_sa'] = ct_dests
    dene['orig_sa'] = ct_origs
    
    return dene


In [ ]:
def euc_dist(ctgan_sample,rnn_sample,ind):
    
    return (
    ((ctgan_sample['xo_c'].iloc[ind] - rnn_sample['xo_0'])**2 ) + ( (ctgan_sample['yo_c'].iloc[ind] - rnn_sample['yo_0'])**2 )
    ) ** (1/2)

def create_frame(ctgan_sample,rnn_sample):
    frame_list=[]
    for i in range(len(ctgan_sample)): 
        a=euc_dist(ctgan_sample,rnn_sample,i).to_numpy()
        frame_list.append(a)
        #distance_matrix=pd.concat([distance_matrix,a],axis=1)
        #if i%100 == 0:
            #print(i)
    frame_list=np.asarray(frame_list)

    return frame_list

### Ust indeksler, yani columns are ct, rowlar sol indeks rnn ler...


In [ ]:

def find_max_ind(merger):
    merg_dist= merger[['distance01','distance02','distance03']].idxmax(axis = 1)
    merger['d_point'] = 0
    merger['o_point'] = 0
    merger['dest_sa'] = 0
    
    for i in range(len(merger)):
        indeks=merg_dist[i]

        if indeks == 'distance01':
            merger['d_point'].iloc[i]= Point( (merger['xo_1'].iloc[i],merger['yo_1'].iloc[i]) )
            merger['o_point'].iloc[i] =Point( (merger['xo_0'].iloc[i],merger['yo_0'].iloc[i]) )
            merger['dest_sa'].iloc[i] = merger['1'].iloc[i]

    
        elif indeks =='distance02':
            merger['d_point'].iloc[i]= Point( (merger['xo_2'].iloc[i],merger['yo_2'].iloc[i]) )
            merger['o_point'].iloc[i] =Point( (merger['xo_0'].iloc[i],merger['yo_0'].iloc[i]) )
            merger['dest_sa'].iloc[i] = merger['2'].iloc[i]


            
        elif indeks=='distance03':
            merger['d_point'].iloc[i]= Point( (merger['xo_3'].iloc[i],merger['yo_3'].iloc[i]) )
            merger['o_point'].iloc[i] =Point( (merger['xo_0'].iloc[i],merger['yo_0'].iloc[i]) )
            merger['dest_sa'].iloc[i] = merger['3'].iloc[i]

        if i%100==0:
            print(str(i))
                
    return merger



In [10]:
#data_loads

qts2=pd.read_excel('/Users/erensmacbook/Downloads/2qts.xlsx')
qts5=pd.read_excel('/Users/erensmacbook/Downloads/5qts.xlsx')

# Set filepath
fp = "/Users/erensmacbook/Downloads/SA1_2021_AUST_SHP_GDA2020/SA1_2021_AUST_GDA2020.shp"

# Read file using gpd.read_file()
data = gpd.read_file(fp)

##generating sa1-sa2 pairs to match them later on in result page
sa1_sa2= data[['SA1_CODE21','SA2_CODE21']][0:-1]
sa1_sa2['SA1_CODE21']=sa1_sa2['SA1_CODE21'].map(int)
sa1_sa2['SA2_CODE21']=sa1_sa2['SA2_CODE21'].map(int)


In [23]:
#process cnn

qts2_t=qts2[['PERSID','AGEGROUP','SEX','WORKSTATUS','INDUSTRY']]
qts2_t=qts2_t[(qts2_t.INDUSTRY.isna() != True)] ## Ogrenci bilgileri icin mainact ve studying later

qts5_t=qts5[['PERSID','TRIPID',
      'ORIGPLACE','ORIGSA1_2021'
     ,'DESTPURP','DESTSA1_2021'
     , 'DURATION','STARTIME']]

real_data= pd.merge(qts2_t,qts5_t, on='PERSID')
real_data


data1= data[(data.SA4_NAME21=='Brisbane - East') |
   (data.SA4_NAME21=='Brisbane - North')
    |
   (data.SA4_NAME21=='Brisbane - West')
    |
   (data.SA4_NAME21=='Brisbane Inner City')
           |
   (data.SA4_NAME21=='Brisbane - South')]

qld_orig = data1[['SA1_CODE21','geometry']]
qld_dest = data1[['SA1_CODE21','geometry']]

qld_orig=qld_orig.rename(columns={"SA1_CODE21":"ORIGSA1_2021"})
qld_dest=qld_dest.rename(columns={"SA1_CODE21":"DESTSA1_2021"})

qld_orig['ORIGSA1_2021']=qld_orig['ORIGSA1_2021'].astype(int)
qld_dest['DESTSA1_2021']=qld_dest['DESTSA1_2021'].astype(int)

qld_orig=qld_orig.rename(columns={"geometry":"geo_o"})
qld_dest=qld_dest.rename(columns={"geometry":"geo_d"})


train_data = pd.merge(real_data,qld_orig, on="ORIGSA1_2021")
train_data= pd.merge(train_data,qld_dest, on="DESTSA1_2021")

train_data=train_data.sort_values(by=['PERSID', 'DURATION'])
train_data= train_data.drop_duplicates(['PERSID'], keep='last').reset_index()

train_data = train_data[['AGEGROUP', 'SEX','INDUSTRY','ORIGSA1_2021','DESTSA1_2021','geo_o','geo_d']]

train_data['xo_c'] = train_data["geo_o"].apply(lambda x: (x.bounds[0]+x.bounds[2]) /2)
train_data['yo_c'] = train_data["geo_o"].apply(lambda x: (x.bounds[1]+x.bounds[3]) /2)

train_data['xd_c'] = train_data["geo_d"].apply(lambda x: (x.bounds[0]+x.bounds[2]) /2)
train_data['yd_c'] = train_data["geo_d"].apply(lambda x: (x.bounds[1]+x.bounds[3]) /2)


train_data = pd.DataFrame(train_data)
train_data


train_cat = train_data[['AGEGROUP','SEX','INDUSTRY','ORIGSA1_2021','DESTSA1_2021']]
train_cord =train_data[['AGEGROUP','SEX','INDUSTRY','xo_c','yo_c','xd_c','yd_c']]

In [35]:
train_cat

,AGEGROUP,SEX,INDUSTRY,ORIGSA1_2021,DESTSA1_2021
0,10,male,utilities,30403109418,30501110537
1,14,male,construction,30101100407,30103101509
2,11,female,professional,30103101601,30103101509
3,11,male,professional,30103101601,30103101509
4,13,female,wholesale,30303106525,30103101509
...,...,...,...,...,...
5351,4,male,other,30103101401,30103101509
5352,9,female,realEstate,30103101505,30103101509
5353,12,female,education,30301105142,30103101509
5354,12,male,transport,30301105142,30103101509


In [ ]:
### save proccessing for traiining
train_cord.to_csv('/Users/erensmacbook/Desktop/train_cord_uniq.csv',index= False)

In [5]:
#ctgan train

'''
!pip install sdv

from google.colab import files
from sdv.constraints.base import Constraint, import_object
from sdv.constraints import FixedCombinations
from sdv.tabular import CTGAN

df1= pd.read_csv(' SAVED DATA HERE ',index_col =None)
from sdv.constraints.base import Constraint, import_object
from sdv.constraints import FixedCombinations
from sdv.tabular import CTGAN

fixed_company_department_constraint = FixedCombinations(
    column_names=['INDUSTRY','xo_c','yd_c'])

types = {
      'INDUSTRY' : { 'type': 'categorical'}  ,    
      'xo_c' : { 'type': 'numerical', "subtype": "float"} ,
      'yo_c' : { 'type': 'numerical', "subtype": "float"} ,
      'xd_c' : { 'type': 'numerical', "subtype": "float"} ,
      'yd_c' : { 'type': 'numerical', "subtype": "float"} }
      

ctgan5 = CTGAN(epochs=400,batch_size=20 ,constraints = [fixed_company_department_constraint],
       field_types=types,discriminator_lr=0.01e-4 , generator_lr=0.01e-4 
              , generator_dim=(32,32,32), discriminator_dim=(32, 32, 32)   )
ctgan5.fit(df1)
data5c = ctgan5.sample(50000)

data5c.to_csv('ctgan50k.csv') 
files.download('ctgan50k.csv')

'''

SyntaxError: invalid syntax (694453659.py, line 1)

In [30]:
train_data_rnn = pd.merge(real_data,qld_orig, on="ORIGSA1_2021")
train_data_rnn = pd.merge(train_data_rnn,qld_dest, on="DESTSA1_2021")
train_data_rnn=train_data_rnn.sort_values(by=['PERSID', 'STARTIME'])
cop=train_data_rnn.copy()

act_ind=cop.PERSID.unique()

group_traject=cop.groupby('PERSID')
#group_traject.get_group(act_ind[2])

In [43]:
a_of_a=[]
industries=[]
for i in act_ind:
    temp=group_traject.get_group(i)
    templist=temp.ORIGSA1_2021.to_list()
    son=temp.DESTSA1_2021.to_list()[-1]
    verdict=temp.INDUSTRY.values[0]
    templist.append(son)
    industries.append(verdict)

    templist=list(map(str, templist)) 
    a_of_a.append(templist)

act_df=pd.DataFrame.from_records(a_of_a)

In [45]:
### RNN TRAIN

''' CODE IN TEXTGENRNN FOR RNN , COLAB FOR GPT CONDITIONAL'''

' CODE IN TEXTGENRNN FOR RNN , COLAB FOR GPT CONDITIONAL'

In [48]:
## Matching CTGAN AND RNN RESULTS

''' download the results from rnn and ctgan'''
# rnn50k= pd.read_csv('/Users/erensmacbook/Downloads/rnn50k.csv',index_col=0)
#ctgan_sample= pd.read_csv('/Users/erensmacbook/Downloads/ctgan50k.csv',index_col=0)

#area_dict = dict(zip(qld_orig.ORIGSA1_2021, qld_orig.geo_o))
#area_dict[0]=0
#area_dict2 ={str(k):v for k,v in area_dict.items()}


#generate_merger(rnn50k)
#add_distance(rnn50k)



### ready for hungarian matching
# pd.concat([rnn_sample,rnn_sample2],ignore_index=True) ## concat ctgan and rnn if files are too big
#splitted_ct=   np.array_split(ctgan_sample_170num, 20)
#splitted_rnn=    np.array_split(rnn_sample, 20)   ## split population and trips for hungarian matching, max15k match per epoch is recommended


## hungarianmatching

#framelist1=create_frame(ctgan_sample,rnn)
#row_ind1, col_ind1 = linear_sum_assignment(framelist1)
#print("LSA1 solved")

#d1=dict(zip(col_ind1,row_ind1)) #keeep indices of best matches
#assigned1=assign_toct(d1,ctgan_sample,rnn) ## assign values cnn and rnn


''' if you have big population as chunks, matching routine for chunks here'''

'''frames=[]

for i in range(len(splitted_ct)):
    frame=create_frame(splitted_ct[i],splitted_rnn[i])
    frames.append(frame)
    print(str(i))

print('ready')
## jıpyter issue?

rowinds=[]
colinds=[]
count=0
for i in frames:
    
    row_ind1, col_ind1 = linear_sum_assignment(i)
    rowinds.append(row_ind1)
    colinds.append(col_ind1)
    count = count + 1
    print(str(count))
print("LSA1 solved")


hung_dicts=[]
for i in range(len(colinds)):
    d1=dict(zip(colinds[i],rowinds[i]))#for ct 1051, rnn 0 is best
    hung_dicts.append(d1)


assignments=[]
for i in range(len(colinds)):
    assigned1=assign_toct(hung_dicts[i],splitted_ct[i],splitted_rnn[i])
    assignments.append(assigned1)
    
    
'''

# assigned_all=pd.concat(assignments, ignore_index=True)
# assigned_all.to_csv('/Users/erensmacbook/Downloads/hungarian200k.csv',index= False)

'frames=[]\n\nfor i in range(len(splitted_ct)):\n    frame=create_frame(splitted_ct[i],splitted_rnn[i])\n    frames.append(frame)\n    print(str(i))\n\nprint(\'ready\')\n## jıpyter issue?\n\nrowinds=[]\ncolinds=[]\ncount=0\nfor i in frames:\n    \n    row_ind1, col_ind1 = linear_sum_assignment(i)\n    rowinds.append(row_ind1)\n    colinds.append(col_ind1)\n    count = count + 1\n    print(str(count))\nprint("LSA1 solved")\n\n\nhung_dicts=[]\nfor i in range(len(colinds)):\n    d1=dict(zip(colinds[i],rowinds[i]))#for ct 1051, rnn 0 is best\n    hung_dicts.append(d1)\n\n\nassignments=[]\nfor i in range(len(colinds)):\n    assigned1=assign_toct(hung_dicts[i],splitted_ct[i],splitted_rnn[i])\n    assignments.append(assigned1)\n    \n    \n'

In [49]:
### showing results plotting correlation etc...


### after matching we need re-process the data 

'''hungariangpt=pd.read_csv('/Users/erensmacbook/Downloads/hungarian-gpt.csv')
hungariangpt=hungariangpt.rename(columns={"dest_sa": "DESTSA1_2021", "orig_sa": "ORIGSA1_2021"})
hungariangpt=generate_distance(hungariangpt)
hungariangpt=add_sa2(hungariangpt) '''


#plot_area_valid(validation,hungariangpt,'education','15k validation health',25)
#plot_area(real_cat2,hungariangpt,'education','15k validation health',25)
#plot_distance_c(real_cat,hungariangpt,'condrnn')

In [50]:
### validation dataset process

'''validation=pd.read_csv('/Users/erensmacbook/Desktop/validation2.csv',index_col=1)
#validation.rename(columns={"Unnamed: 1":"ORIGSA1_2021"})
validation=validation.rename(columns={" ":"SA2_CODE21"})
validation = validation.iloc[1: , :]

col_list=validation.columns.to_list()
col_list=col_list[1:-1]

col_list2=validation.columns.to_list()
col_list2=col_list2[1:23]

for i in col_list2:
    validation[i] = validation[i].apply(lambda x:0 if x==' np ' else x)

for i in col_list2:
    validation[i] = validation[i].astype(float)

for i in col_list:
    validation[i] = validation[i]/validation['Total']

validation['SA2_CODE21']=validation['SA2_CODE21'].map(int)
validation=validation.rename(columns={"SA2_CODE21":"dest_sa2"})'''

'validation=pd.read_csv(\'/Users/erensmacbook/Desktop/validation2.csv\',index_col=1)\n#validation.rename(columns={"Unnamed: 1":"ORIGSA1_2021"})\nvalidation=validation.rename(columns={" ":"SA2_CODE21"})\nvalidation = validation.iloc[1: , :]\n\ncol_list=validation.columns.to_list()\ncol_list=col_list[1:-1]\n\ncol_list2=validation.columns.to_list()\ncol_list2=col_list2[1:23]\n\nfor i in col_list2:\n    validation[i] = validation[i].apply(lambda x:0 if x==\' np \' else x)\n\nfor i in col_list2:\n    validation[i] = validation[i].astype(float)\n\nfor i in col_list:\n    validation[i] = validation[i]/validation[\'Total\']\n\nvalidation[\'SA2_CODE21\']=validation[\'SA2_CODE21\'].map(int)\nvalidation=validation.rename(columns={"SA2_CODE21":"dest_sa2"})'

In [ ]:
### correlation analysis for diffferent methods

'''
sector='admin'
hungariandest=hungarian200k.dest_sa2.unique()
realdest=real_cat2.dest_sa2.unique()
validdest=validation.dest_sa2.unique()
#indices=np.intersect1d(hungariandest,validdest)
indices=realdest

hungarian_educ=[]
realdest_educ=[]
valid_educ=[]
gpt_educ=[]
before_educ=[]
#ctgan_stringmatch=[]


ctgan_states = hungarian200k.groupby(['dest_sa2','INDUSTRY'])['INDUSTRY'].count()
    # Change: groupby state_office and divide by sum
ctgan_pcts = ctgan_states.groupby(level=0).apply(lambda x:
                                                 100 * x / float(x.sum()))

real_states = real_cat2.groupby(['dest_sa2','INDUSTRY'])['INDUSTRY'].count()
    # Change: groupby state_office and divide by sum
real_pcts = real_states.groupby(level=0).apply(lambda x:
                                                 100 * x / float(x.sum()))

gpt_states = hungariangpt.groupby(['dest_sa2','INDUSTRY'])['INDUSTRY'].count()
    # Change: groupby state_office and divide by sum
gpt_pcts = gpt_states.groupby(level=0).apply(lambda x:
                                                 100 * x / float(x.sum()))

before_states = ctgan_before.groupby(['dest_sa2','INDUSTRY'])['INDUSTRY'].count()
    # Change: groupby state_office and divide by sum
before_pcts = before_states.groupby(level=0).apply(lambda x:
                                                 100 * x / float(x.sum()))

for i in indices:
    try:
        hungarian_educ.append(ctgan_states[i][sector])
    except KeyError:
        hungarian_educ.append(0)
        
for i in indices:
    try:
        realdest_educ.append(real_states[i][sector])
    except KeyError:
        realdest_educ.append(0)
        
for i in indices:
    try:
        #print(validation[validation.dest_sa2==i][sector][0])
        if math.isnan(validation_n[validation_n.dest_sa2==i][sector][0]) == True:
            valid_educ.append(0)
        else:
            valid_educ.append(validation_n[validation_n.dest_sa2==i][sector][0]*100)
    except KeyError:
        valid_educ.append(0)
    except IndexError:
        valid_educ.append(0)

for i in indices:
    try:
        gpt_educ.append(gpt_states[i][sector])
    except KeyError:
        gpt_educ.append(0)
        
for i in indices:
    try:
        before_educ.append(before_states[i][sector])
    except KeyError:
        before_educ.append(0)


'''

'''df = pd.DataFrame()
mySeries = pd.Series(realdest_educ) 
mySeries2 = pd.Series(hungarian_educ) 
mySeries3 = pd.Series(valid_educ) 
mySeries4 = pd.Series(gpt_educ) 
mySeries5 = pd.Series(before_educ) 

df['samp']=mySeries
df['hungarian']=mySeries2
df['valid']=mySeries3
df['cond-rnn']=mySeries4
df['ct-before']=mySeries5'''


'''import pingouin as pg
#pg.corr(x=df['A'], y=df['C'],method='spearman')
pg.pairwise_corr(df,method='spearman') 
# import numpy as np
from sklearn.metrics import pairwise_distances
from scipy.spatial.distance import cosine
from scipy import spatial

## cosine similiartiy

result = 1 - spatial.distance.cosine(mySeries, mySeries3)
print('cosine similarity of valid vs sample: ' + str(result))

result = 1 - spatial.distance.cosine(mySeries2, mySeries)
print('cosine similarity of hungarian vs sample: ' + str(result))

result = 1 - spatial.distance.cosine(mySeries2, mySeries3)
print('cosine similarity of hungarian vs valid: ' + str(result))

result = 1 - spatial.distance.cosine(mySeries4, mySeries)
print('cosine similarity of cond vs sample: ' + str(result))

result = 1 - spatial.distance.cosine(mySeries4, mySeries3)
print('cosine similarity of cond vs valid: ' + str(result))

result = 1 - spatial.distance.cosine(mySeries5, mySeries)
print('cosine similarity of ct-before vs sample: ' + str(result))

result = 1 - spatial.distance.cosine(mySeries5, mySeries3)
print('cosine similarity of ct-before vs valid: ' + str(result)) '''


### plotting correlation

'''g = sns.JointGrid(data=df, x='samp', y='valid', height=5.5)
g = g.plot_joint(sns.regplot, color="xkcd:muted blue")
g = g.plot_marginals(sns.distplot, kde=False, bins=10, color="xkcd:bluey grey")
#g.ax_joint.text(1,5,'r = 0.45, p < .001', fontstyle='italic')
#plt.tight_layout()
'''